# Valuing American Options by Simulation: A Simple Least-Square Approach

Implementation by Python

For an American option, the optimal strategy to exercise is to compare the immediate exercise value with the expected cash flows from continuing. Thus, The key to optimally exercising an American option is identifying the conditional expected value of continuation.


$ dS = \mu Sdt + \sigma SdZ $

The formula for calculating stock price at time t is:

$ S_{t_{i+1}} = S_{t_{i}} e^{(\mu - \frac{1}{2}\sigma^2)(t_{i+1}-t_{i})+\sigma \sqrt{t_{i+1}-t_{i}} Z_{i+1}} $,
where $Z \sim N(0, 1)$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D

In [2]:
'''    
parameter description:
S0: initial price
n: number of steps
t: starting time
T: terminating time
St: trajectory of price
a, b: the first and second moment of log Y
d: dividend yield
'''

def Geometric_Brownian_Motion_Trajectory( mu, sigma, S0, n, t, T ): 
    
    time = np.linspace(t, T, n + 1) 
    delta_time = time[1] - time[0] 
    St = np.zeros(n + 1)
    St[0] = S0
    z = np.random.standard_normal(n) 
    for i in range(n):
        St[i + 1] = St[i] * np.exp((mu - 1 / 2 * sigma ** 2) * delta_time + sigma * delta_time ** (1 / 2) * z[i])
    return St

def Geometric_Brownian_Motion_Jump( mu, sigma, d, S0, n, t, T, a, b, lam ):
    
    delta_t = (T - t) / n
    St = np.zeros(n + 1)
    X = np.zeros(n + 1)
    z = np.random.normal(size=(n + 1, 1))
    X[0] = np.log(S0)
    for i in range(1, n + 1):
        n = np.random.poisson(lam * delta_t)
        if n == 0:
            m =0 
        else:
            m = a * n + b * n ** 0.5 * np.random.normal()
        X[i] = X[i - 1] + (mu - d - 0.5 * sigma ** 2) * delta_t + sigma * delta_t ** 0.5 * z[i] + m
        St = np.exp(X)
    return St


In [3]:
def Black_Scholes_European_Put( t, T, St, K, r, d, vol): 
    
    d1 = 1 / (vol * (T-t)**(1/2)) * (np.log(St/K) + (r - d + 1/2 * vol**2) * (T-t))
    d2 = d1 - vol * (T-t)**(1/2)
    norm1 = norm.cdf(-d1)
    norm2 = norm.cdf(-d2)
    bs_european_put_price = -np.exp(-d*(T-t)) * St * norm1 + np.exp(-r*(T-t)) * K * norm2

    return bs_european_put_price


In [4]:
def Black_Scholes_Implicit_FD_EO( K, r, d, vol, Smin, Smax, t, T, N, M, initial_condition, boundary_condition ):
        
    delta_s = (Smax-Smin)/N
    delta_tao = (T-t)/M
        
    if delta_tao / (delta_s)**2 < 0 or delta_tao / (delta_s)**2 >= 1/2:
        print( 'stability condition does not hold.' )
        quit()
            
    else:
        St = np.linspace(Smin, Smax, N+1)
        tao = np.linspace(t, T, M+1)
        v = np.zeros((N+1, M+1))  # option value array
            
        # Calculating the weighting matrix
        lI = -(St**2 * vol**2 * delta_tao) / (2 * (delta_s)**2) + ((r-d) * St * delta_tao) / (2 * delta_s)
        dI = 1 + r*delta_tao + vol**2 * St**2 * delta_tao / (delta_s)**2
        uI = -vol**2 * St**2 * delta_tao / (2 * (delta_s)**2) - (r-d) * St * delta_tao / (2 * delta_s) 
        
        wm = np.zeros((N+1, N+1))
        for i in range(1, N):
            wm[i, i-1] = lI[i]
            wm[i, i] = dI[i]
            wm[i, i+1] = uI[i]
            
        wm[0, 0] = 2*lI[0] + dI[0]
        wm[0, 1] = uI[0] - lI[0]
        wm[N, N-1] = lI[N] - uI[N]
        wm[N, N] = dI[N] + 2*uI[N]
            
        # calculate the price
        if initial_condition == 'ic_call':
            v[:, 0] = ((St > K) + 0.0) * (St - K)
        elif initial_condition == 'ic_put':
            v[:, 0] = ((St < K) + 0.0) * (K - St)
        else:
            print( 'initial condition is not appropriate!' )
            
        if boundary_condition == 'dirichlet_bc':
            for k in range(1, M+1):
                v[:, k] = np.linalg.inv(wm) @ (v[:, k-1])
                v[0, k] = v[0, k-1]
                v[N, k] = v[N, k-1]
        elif boundary_condition == 'neumann_bc':
            for k in range(1, M+1):
                v[:, k] = np.linalg.inv(wm) @ v[:, k-1]      
        else:
            print( 'boundary condition is not appropriate!' )

    bs_implicit_fd_eo_price = v
    return bs_implicit_fd_eo_price



In [5]:
def Valuation_by_Least_Square( r, sigma, S0, m, n, t, T ):
    
    # Create m paths of stock price with n steps of time by simulation
    St_GeoBro = np.zeros((m, n+1))
    for i in range(m):
        St_GeoBro[i, :] = Geometric_Brownian_Motion_Trajectory( r, sigma, S0, n, t, T )

    # Payoff is a matrix of the amount of cash flow at each step if immediately exercising the option,
    # which is only for convenience of calculation in the following procedures 
    Payoff = np.maximum( K - St_GeoBro, 0 )
        
    # Cash_Flow is a matrix similar to Payoff, 
    # which is updated by doing regression and deciding whether to exercise immediately
    Cash_Flow = np.maximum( K - St_GeoBro, 0 )

    # Calculate the conditional expected value of continuation
    # 1. regressing (Y = the discounted payoff at time t_i+1) against (X = the stock price, whose option is in the money at time t_i) 
    # 2. predict the expected conditional value at time t_i by substituing the basis functions of X into the regression formula
    # 3. compare the expected conditional value with the immediate value
    # 4. if the immediate value is greater, exercise immediately

    for i in range(n-1):
    
        # X is the payoff if exercise in the money at time t_i
        X = ( Payoff[:, n-1-i] )[ Payoff[:, n-1-i] > 0 ]
        if X.size == 0:
            continue
        
        # Y is the discounted payoff at time t_i+1, related to X
        Y = ( Payoff[:, n-i] )[ Payoff[:, n-1-i] > 0 ] * np.exp( -r * 1/n * (i+1) )

        # L0, L1, L2 are basis functions of X
        # combine them into a single matrix for following regression 
        X = X.reshape(np.size(X),1)
        L0 = np.exp( -X/2 )
        L1 = L0 * ( 1 - X )
        L2 = L0 * ( 1 - 2*X + X**2/2 )
        XX = np.hstack((L0, L1, L2))

        # regress Y ~ intercept + a * L0 + b * L1 + c * L2
        reg = LinearRegression().fit(XX, Y)
        # calculate the predicted value of Y (i.e. the conditional expected value of continuation)
        Y_predict = reg.predict(XX)
        Y_predict = Y_predict.reshape(1,np.size(Y_predict))
        # compare the immediate exercise value with the conditional expected value of continuation
        # and decide whether to exercise immediately
        exercise = ( Y_predict < Payoff[ Payoff[:, n-1-i] > 0, n-1-i ] ) * Payoff[ Payoff[:, n-1-i] > 0, n-1-i ]

        # substitue those values decided to exercise immediately into the Cash_Flow matrix
        # and set the continuing values of Cash_Flow to zero, as the option is exercised obly once
        Cash_Flow[ Payoff[:, n-1-i] > 0, n-1-i ] = exercise
        Cash_Flow[ Payoff[:, n-1-i] > 0, n-i: ] = 0
    
    # calculate the discounted factor matrix
    df = np.ones( np.shape( Cash_Flow[:, 1:] ) )
    for i in range(n):
        df[:, i] = np.exp( -r * 1/n * (i+1) )
    
    # calculate the present value of each path
    PV_of_Cash_Flow = np.sum( (Cash_Flow[:, 1:] * df), axis=1 )
    
    # calculate the value of the option
    # by averaging the value of each presen value of each path
    value = np.mean(PV_of_Cash_Flow)
    
    return value


In [6]:
S0 = 36 #initial price
K = 40 #strike price
r = 0.06 #short-term interest rate
sigma = 0.2 #volatility
t = 0 #initial time
T = 1 #maturity time
d = 0 #dividend yield

n = 50 # 50 steps per year
m = 1000 # 100000 paths of stock price

In [7]:
Value_LSM = Valuation_by_Least_Square( r, sigma, S0, m, n, t, T )
Value_LSM

3.8925499492647297

In [8]:
Value_BS = Black_Scholes_European_Put( t, T, S0, K, r, d, sigma )
Value_BS

3.84430779159684

In [9]:
N = 100 # 1000 steps for the stock price
M = 4000 # 40000 time steps per year
Smin = 1
Smax = 100

In [10]:
p = Black_Scholes_Implicit_FD_EO( K, r, d, sigma, Smin, Smax, t, T, N, M, 'ic_put', 'neumann_bc' )

In [11]:
S0_list = [36, 38, 40, 42, 44]
sigma_list = [0.2, 0.4]
T_list = [1, 2]

In [12]:
# create a dataframe for the final results
df = pd.DataFrame(np.zeros((20, 10)), columns=['$S$', '$\sigma$', '$T$', 'Finite Difference American', 'Closed Form European', 'Early Exercise Value 1', 'Simulated American', 'Closed Form European', 'Early Exercise Value 2', 'Difference in early exercise value'], dtype=float)

In [13]:
df

,$S$,$\sigma$,$T$,Finite Difference American,Closed Form European,Early Exercise Value 1,Simulated American,Closed Form European,Early Exercise Value 2,Difference in early exercise value
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# do calculation for all the situations and put values into the table
i = 0
for S0 in S0_list:
    for sigma in sigma_list:
        for T in T_list:
            Value_LSM = Valuation_by_Least_Square( r, sigma, S0, m, n, t, T )
            Value_BS = Black_Scholes_European_Put( t, T, S0, K, r, d, sigma )
            p = Black_Scholes_Implicit_FD_EO( K, r, d, sigma, Smin, Smax, t, T, N, M, 'ic_put', 'neumann_bc' )
            Value_FD = p[int(S0/(Smax - Smin)*N), -1]
            
            df.loc[i, '$S$'] = S0
            df.loc[i, '$\sigma$'] = sigma
            df.loc[i, '$T$'] = T
            
            df.loc[i, 'Finite Difference American'] = Value_FD
            df.loc[i, 'Closed Form European'] = Value_BS
            df.loc[i, 'Early Exercise Value 1'] = Value_FD - Value_BS
            
            df.loc[i, 'Simulated American'] = Value_LSM
            df.loc[i, 'Early Exercise Value 2'] = Value_LSM - Value_BS
            df.loc[i, 'Difference in early exercise value'] = (Value_FD - Value_BS) - (Value_LSM - Value_BS)

            i = i+1
            if i == 20:
                break

In [15]:
df

,$S$,$\sigma$,$T$,Finite Difference American,Closed Form European,Early Exercise Value 1,Simulated American,Closed Form European,Early Exercise Value 2,Difference in early exercise value
0,36.0,0.2,1.0,3.502825,3.844308,-0.341482,3.944338,3.844308,0.100030,-0.441513
1,36.0,0.2,2.0,3.499024,3.763001,-0.263977,3.901384,3.763001,0.138383,-0.402360
2,36.0,0.4,1.0,6.418688,6.711399,-0.292711,3.793793,6.711399,-2.917607,2.624895
3,36.0,0.4,2.0,7.459062,7.700040,-0.240978,3.835473,7.700040,-3.864567,3.623589
4,38.0,0.2,1.0,2.586434,2.851932,-0.265498,1.841548,2.851932,-1.010384,0.744886
5,38.0,0.2,2.0,2.778983,2.990557,-0.211574,1.761463,2.990557,-1.229093,1.017520
6,38.0,0.4,1.0,5.583256,5.834321,-0.251065,2.010724,5.834321,-3.823597,3.572532
7,38.0,0.4,2.0,6.766214,6.978802,-0.212588,2.164013,6.978802,-4.814789,4.602201
8,40.0,0.2,1.0,1.867044,2.066401,-0.199357,0.379878,2.066401,-1.686523,1.487166
9,40.0,0.2,2.0,2.188857,2.355866,-0.167009,0.637795,2.355866,-1.718071,1.551062
